# Data Loader Circuits

Many quantum algorithms require the conversion of classical data into quantum states. QC Ware has developed carefully optimized NISQ circuits that perform this translation.

This notebook explains how to generate these circuits for your own data sets.

### Basic Usage

In [1]:
from qcware import forge
# this line is for internal tracking; it is not necessary for use!
forge.config.set_environment_source_file('data_loaders.ipynb')

# Paste your api key below if not using the hosted notebooks# qcware.config.set_api_key('PASTE YOUR API KEY')
import numpy as np
from qcware.forge import qio

In [2]:
# Generate a random vector
data = np.random.rand(4)-.5
normalized_data = data / np.linalg.norm(data)

print('Generated data:')
print(normalized_data)

Generated data:
[-0.40626352 -0.75951554 -0.49535081 -0.11275491]


In [3]:
# Create the data loader circuit for the data
data_loader = qio.loader(normalized_data)

print(data_loader)

T  : |0|1|2|3|4|5|

q0 : -X-B-@-------
        | |       
q1 : ---S-|---@---
          |   |   
q2 : -----X-B-X-B-
            |   | 
q3 : -------S---S-
                  
T  : |0|1|2|3|4|5|



We can look at how the data is stored. You should be able to find the normalized data in the quantum state.

In [4]:
# Simulate the data loader circuit
import quasar
state_vector = quasar.QuasarSimulatorBackend().run_statevector(data_loader)

print(state_vector)

[ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j -0.11275491+0.j -0.49535081+0.j  0.        +0.j
  0.        +0.j -0.75951554+0.j -0.40626352+0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]


### Loader Type Selection

There are two types of loaders that we have designed: parallel loaders (which have less depth but more qubits) and optimized loaders (which have fewer qubits with greater depth).

In [5]:
data = np.random.rand(5)
normalized_data = data / np.linalg.norm(data)

parallel_loader = qio.loader(normalized_data, mode='parallel')
optimized_loader = qio.loader(normalized_data, mode='optimized')

In [6]:
print('Parallel Loader (more qubits, less depth):\n')
print(parallel_loader, '\n\n')

print('Optimized Loader (fewer qubits, greater depth):\n')
print(optimized_loader)

Parallel Loader (more qubits, less depth):

T  : |0|1|2|3|

q0 : -X-B-B-B-
        | | | 
q1 : ---|-|-S-
        | |   
q2 : ---|-S-B-
        |   | 
q3 : ---|---S-
        |     
q4 : ---S-----
              
T  : |0|1|2|3|
 


Optimized Loader (fewer qubits, greater depth):

T  : |0|1|2|3|4|5|6|7|8|

q0 : -X-B-@-------------
        | |             
q1 : ---S-|-------@-----
          |       |     
q2 : -----X-B-B-B-X-B-B-
            | | |   | | 
q3 : -------|-S-|---|-S-
            |   |   |   
q4 : -------S---S---S---
                        
T  : |0|1|2|3|4|5|6|7|8|



### Running Loaders on Real Hardware (Ion Q)

For a realistic application, a user would first use a loader and then add additional gates to implement a quantum algorithm. However, for simplicity we will just demonstrate running and measuring a loader circuit by itself.

In [7]:
# Generate some data
data = np.random.rand(4)
normalized_data = data / np.linalg.norm(data)

# Make a loader circuit
loader = qio.loader(normalized_data, mode='optimized')

# Get an Ion Q backend 
ion_q_backend = forge.circuits.QuasarBackend('awsbraket/ionq')

# Uncomment this line to perform hardware run:
# result = ion_q_backend.run_measurement(circuit=loader, nmeasurement=64)

Note that the result of a physical hardware run may be delayed because of hardware availability. You can retrieve the result using the API tab on Forge when the computation has been completed.